In [1]:
import requests
import os,re,json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
pd.set_option("max_colwidth",1000000)
pd.set_option('max_columns', 50)

# Gather

Create a data frame twitter_archive

In [5]:
url="https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv"
twitter_archive=pd.read_csv(url)
twitter_archive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None


In [4]:
twitter_archive.shape

(2356, 17)

Create a data frame images

In [6]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open(url.split('/')[-1], mode='wb') as file:
    file.write(response.content)
    
images = pd.read_csv('image-predictions.tsv', sep='\t')
images.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [6]:
images.shape

(2075, 12)

Merge twitter_archive and images by tweet_id

In [7]:
archive_images=pd.merge(twitter_archive, images, how='outer',on='tweet_id')
archive_images.shape

(2356, 28)

Search by tweed_id and retrieve favorite_count and retweet_count

In [2]:
import tweepy

consumer_key = 	'F0wLeqL2m6XSMl8DWU6S7mlJm'
consumer_secret = 'PSYpGQHbDcxWmHLQvp5XODKL4wUVdddvl00mL99C1wk58Pnau3'

access_token = '106335650-XVaCgWgxQwwjW0vZH5pAftO6CwKJr2Gz3824NAsI'
access_token_secret = 'usQWjVz2pujFZFjPJdIDCPB6lXTApYRbmLZjAOAVjvKEy'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [11]:
tweet_list=[]


for tweet_id in archive_images['tweet_id']:
    try:
        tweet_status=api.get_status(tweet_id)._json
        favorite_count = tweet_status['favorite_count']
        retweet_count = tweet_status['retweet_count']
        tweet_list.append({'tweet_id': int(tweet_id),'favorite_count': int(favorite_count),'retweet_count': int(retweet_count)})
    
    except Exception:  
        print("error ids:  " +str(tweet_id))
        tweet_list.append({'tweet_id': int(tweet_id)})

error ids:  888202515573088257
error ids:  873697596434513921
error ids:  869988702071779329
error ids:  861769973181624320
error ids:  842892208864923648
error ids:  802247111496568832
error ids:  775096608509886464


In [12]:
tweet_table = pd.DataFrame(tweet_list, columns = ['tweet_id', 'favorite_count', 'retweet_count'])

In [13]:
tweet_table.to_csv('tweet_table.csv')

In [14]:
tweet_table=pd.read_csv('tweet_table.csv')
tweet_table.head(10)


,Unnamed: 0,tweet_id,favorite_count,retweet_count
0,0,892420643555336193,39306.0,8778.0
1,1,892177421306343426,33661.0,6428.0
2,2,891815181378084864,25358.0,4268.0
3,3,891689557279858688,42684.0,8863.0
4,4,891327558926688256,40841.0,9645.0
5,5,891087950875897856,20474.0,3213.0
6,6,890971913173991426,12012.0,2126.0
7,7,890729181411237888,66395.0,19421.0
8,8,890609185150312448,28103.0,4366.0
9,9,890240255349198849,32344.0,7617.0


In [15]:
tweet_table[tweet_table['favorite_count']==0]

,Unnamed: 0,tweet_id,favorite_count,retweet_count
32,32,886054160059072513,0.0,106.0
36,36,885311592912609280,0.0,19088.0
68,68,879130579576475649,0.0,7079.0
73,73,878404777348136964,0.0,1333.0
74,74,878316110768087041,0.0,6872.0
78,78,877611172832227328,0.0,82.0
91,91,874434818259525634,0.0,15324.0
97,97,873337748698140672,0.0,1642.0
101,101,872668790621863937,0.0,31.0
109,109,871166179821445120,0.0,5925.0


In [16]:
tweet_table.describe

<bound method NDFrame.describe of       Unnamed: 0            tweet_id  favorite_count  retweet_count
0              0  892420643555336193         39306.0         8778.0
1              1  892177421306343426         33661.0         6428.0
2              2  891815181378084864         25358.0         4268.0
3              3  891689557279858688         42684.0         8863.0
4              4  891327558926688256         40841.0         9645.0
5              5  891087950875897856         20474.0         3213.0
6              6  890971913173991426         12012.0         2126.0
7              7  890729181411237888         66395.0        19421.0
8              8  890609185150312448         28103.0         4366.0
9              9  890240255349198849         32344.0         7617.0
10            10  890006608113172480         30993.0         7526.0
11            11  889880896479866881         28103.0         5084.0
12            12  889665388333682689         38570.0         8445.0
13            

In [17]:
tweet_table['favorite_count'].value_counts()

0.0        174
5161.0       3
1376.0       3
2279.0       3
3573.0       3
3494.0       3
315.0        3
1841.0       3
2024.0       3
1254.0       3
912.0        3
2245.0       3
1067.0       3
467.0        3
447.0        3
2434.0       3
3719.0       3
2889.0       2
17372.0      2
3832.0       2
1709.0       2
2427.0       2
1491.0       2
1173.0       2
538.0        2
28103.0      2
1653.0       2
770.0        2
1753.0       2
193.0        2
          ... 
57784.0      1
7058.0       1
19922.0      1
9018.0       1
3539.0       1
39298.0      1
419.0        1
131.0        1
23098.0      1
2251.0       1
466.0        1
46755.0      1
7051.0       1
306.0        1
39996.0      1
12040.0      1
20952.0      1
8788.0       1
2870.0       1
16393.0      1
2383.0       1
1671.0       1
25090.0      1
2058.0       1
31732.0      1
10593.0      1
14194.0      1
10099.0      1
23379.0      1
117.0        1
Name: favorite_count, Length: 2004, dtype: int64

In [18]:
tweet_json = pd.merge(archive_images, tweet_table, how='outer',on='tweet_id')


In [19]:
tweet_json.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,Unnamed: 0,favorite_count,retweet_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False,0,39306.0,8778.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,1,33661.0,6428.0


In [28]:
tweet_json.to_csv('tweet_json.txt')

# Assess

In [3]:
tweet_json=pd.read_csv('tweet_json.txt')

In [30]:
tweet_json.head(5)

,Unnamed: 0,Unnamed: 0.1,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,Unnamed: 0.1.1,favorite_count,retweet_count
0,0,0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False,0,39306.0,8778.0
1,1,1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,1,33661.0,6428.0
2,2,2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1.0,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,2,25358.0,4268.0
3,3,3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1.0,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,3,42684.0,8863.0
4,4,4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2.0,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,4,40841.0,9645.0


In [23]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 32 columns):
Unnamed: 0                    2356 non-null int64
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         235

In [24]:
tweet_json.describe()

,Unnamed: 0,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator,img_num,p1_conf,p2_conf,p3_conf,Unnamed: 0.1,favorite_count,retweet_count
count,2356.000000,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000,2075.000000,2075.000000,2.075000e+03,2.075000e+03,2356.000000,2349.000000,2349.000000
mean,1177.500000,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433,1.203855,0.594548,1.345886e-01,6.032417e-02,1177.500000,8106.075351,3094.855258
std,680.262939,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237,0.561875,0.271174,1.006657e-01,5.090593e-02,680.262939,12048.901960,5129.976423
min,0.000000,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000,1.000000,0.044333,1.011300e-08,1.740170e-10,0.000000,0.000000,0.000000
25%,588.750000,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000,1.000000,0.364412,5.388625e-02,1.622240e-02,588.750000,1414.000000,615.000000
50%,1177.500000,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000,1.000000,0.588230,1.181810e-01,4.944380e-02,1177.500000,3585.000000,1444.000000
75%,1766.250000,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000,1.000000,0.843855,1.955655e-01,9.180755e-02,1766.250000,10090.000000,3604.000000
max,2355.000000,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000,4.000000,1.000000,4.880140e-01,2.734190e-01,2355.000000,131748.000000,78704.000000


In [25]:
tweet_json['name'].value_counts(dropna=False)

None           745
a               55
Charlie         12
Cooper          11
Oliver          11
Lucy            11
Lola            10
Penny           10
Tucker          10
Winston          9
Bo               9
Sadie            8
the              8
an               7
Toby             7
Buddy            7
Daisy            7
Bailey           7
Scout            6
Jax              6
Oscar            6
Dave             6
Stanley          6
Leo              6
Rusty            6
Koda             6
Milo             6
Jack             6
Bella            6
Gus              5
              ... 
Bode             1
Meatball         1
Brandi           1
Smiley           1
Chubbs           1
my               1
Lillie           1
Jazzy            1
Snicku           1
Cheesy           1
Pippin           1
Autumn           1
Richie           1
Venti            1
Maya             1
Baron            1
Jaspers          1
Bilbo            1
Kara             1
Bobb             1
Rooney           1
Billl       

In [26]:
tweet_json['favorite_count'].isnull().value_counts()

False    2349
True        7
Name: favorite_count, dtype: int64

### Quality

1. The type of timestamp is an object rather than time format. 

2. The type of tweet_id and in_reply_to_status_id is numeric rather than string.

3. The type of in_reply_to_user_id is numeric rather than string. There are two tweet_id  appearing in the column in_reply_to_user_id which is for user_id only.

4. The column source contains the whole tag instead of the text.

5. There are emoji characters e.g. 🎶 inside the column text.The unicode character may not correctly be presented in other editor.

6. The type of retweeted_status_id is numeric rather than string.

7. The type of retweeted_status_user_id is numeric rather than string. There are three tweet_id appearing in the column retweeted_status_user_id which is for user_id only.

8. The rating_numerator can be any positive integer, similarly the rating_denominator can be any positive integer out of 10. In fact, any number is possible because it is not a strict technical rating. There are a couple of records with a wrong rating because the author parsed the first fraction rather than the last. There are some tweets without a rating but the author mistakenly parsed a rating for it. There are some retweets back and forth upgrading their previous rating, of which those retweets do not attach links to the picture of the dog they are talking about. It's hard to track back that dog and merge these retweets with the previous rating tweets, I'd like to delete the retweets that is discussing the rating without a dog picture.

9. The name column has meaningless dog name, like 'a', 'an', either the dog name does not exist in the text column or failure to retrieve the dog name. Also, @dog_rates can mispell a dog's name, e.g. "Johm".

10. The favorite_count column has multiple 0 counts but with large number of retweet count, it's almost impossible except for the 'Like' button is disabled. There are 7 records with missing favorite_count and retweet_count because the status does not exist for that tweet.

### Tidiness

1. The column of expanded_urls contains records with multiple links, some records have duplicated links splitted by a comma. The reason is that a grid photo was tweeted, so multiple pictures being recorded, but only one of the sub-photo was use for image prediction. 

2. The columns of doggo floofer pupper puppo are all dog stages, converting these four columns into a stage column should be considered.
3. There should be a column that prints the final breed prediction by summarizing p1 p2 p3 and p1_dog p2_dog p3_dog

# Clean

Make a copy of tweet_json.

In [27]:
tweet_json=pd.read_csv(r'tweet_json.csv')
tweet_json_cleaned=tweet_json.copy()

Change the type of timestamp and retweeted_status_timestamp

Change the type of tweet_id in_reply_to_status_id in_reply_to_user_id retweeted_status_id retweeted_status_user_id

In [28]:
tweet_json_cleaned['timestamp'] = pd.to_datetime(tweet_json_cleaned['timestamp'],utc=True)
tweet_json_cleaned['retweeted_status_timestamp'] = pd.to_datetime(tweet_json_cleaned['retweeted_status_timestamp'],utc=True)

tweet_json_cleaned['tweet_id'] = tweet_json_cleaned['tweet_id'].astype(int).astype(str)

tweet_json_cleaned['in_reply_to_status_id']=tweet_json_cleaned['in_reply_to_status_id'].apply(lambda x : str(int(x)) if not np.isnan(x) else x)
tweet_json_cleaned['in_reply_to_user_id'] = tweet_json_cleaned['in_reply_to_user_id'].apply(lambda x : str(int(x)) if not np.isnan(x) else x)
tweet_json_cleaned['retweeted_status_id'] = tweet_json_cleaned['retweeted_status_id'].apply(lambda x : str(int(x)) if not np.isnan(x) else x)
tweet_json_cleaned['retweeted_status_user_id'] = tweet_json_cleaned['retweeted_status_user_id'].apply(lambda x : str(int(x)) if not np.isnan(x) else x)
tweet_json_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 33 columns):
Unnamed: 0                    2356 non-null int64
Unnamed: 0.1                  2356 non-null int64
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null object
in_reply_to_user_id           78 non-null object
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null object
retweeted_status_user_id      181 non-null object
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper               

Extract text from column source.

Convert doggo floofer pupper puppo into a stage variable.

In [29]:
tweet_json_cleaned['stage']=tweet_json_cleaned['doggo']+tweet_json_cleaned['floofer']+tweet_json_cleaned['pupper']+tweet_json_cleaned['puppo']

In [30]:
def concat_stage(x):
    temp=re.sub('None','',x)
    if temp=="" :
        return 'None'
    else:
        return temp

tweet_json_cleaned['stage']=tweet_json_cleaned['stage'].apply(lambda x : concat_stage(x))

In [31]:
print(tweet_json_cleaned['stage'].value_counts())

None            1976
pupper           245
doggo             83
puppo             29
doggopupper       12
floofer            9
doggofloofer       1
doggopuppo         1
Name: stage, dtype: int64


In [32]:
tweet_json_cleaned['source']=tweet_json_cleaned['source'].apply(lambda x : BeautifulSoup(x, 'html.parser').get_text())


In [33]:
print(tweet_json_cleaned['source'].value_counts())


Twitter for iPhone     2221
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source, dtype: int64


Create a breed_predict column.

In [34]:
for index, row in tweet_json_cleaned.iterrows():
    if row['p1_dog'] == True:
        tweet_json_cleaned.set_value(index,'breed_predict',row['p1'])
        
    elif row['p2_dog'] == True:
        tweet_json_cleaned.set_value(index,'breed_predict',row['p2'])
        
    elif row['p3_dog'] == True:
        tweet_json_cleaned.set_value(index,'breed_predict',row['p3'])


tweet_json_cleaned['breed_predict'].value_counts()

golden_retriever                  173
Labrador_retriever                113
Pembroke                           96
Chihuahua                          95
pug                                65
toy_poodle                         52
chow                               51
Samoyed                            46
Pomeranian                         42
malamute                           34
cocker_spaniel                     34
French_bulldog                     32
Chesapeake_Bay_retriever           31
miniature_pinscher                 26
Cardigan                           23
Staffordshire_bullterrier          22
Eskimo_dog                         22
beagle                             21
German_shepherd                    21
Shih-Tzu                           20
Siberian_husky                     20
Lakeland_terrier                   19
Rottweiler                         19
Shetland_sheepdog                  19
Maltese_dog                        19
kuvasz                             19
basset      

For those tweets having 0 favorite_count, retrieve favorite_count from the retweet_status instead.

In [35]:
for index,row in tweet_json_cleaned.iterrows():
    if row['favorite_count']==0:
        try:
            tweet_status=api.get_status(int(row['tweet_id']))._json
            favorite_count = tweet_status['retweeted_status']['favorite_count']
            tweet_json_cleaned.set_value(index,'favorite_count',favorite_count)

            
        except Exception:  
            print("error ids:  " +row['tweet_id'])

  
        

In [36]:
tweet_json_cleaned['favorite_count'].value_counts(dropna=False)

NaN         7
 1067.0     3
 1376.0     3
 2245.0     3
 912.0      3
 467.0      3
 3494.0     3
 2279.0     3
 5161.0     3
 3719.0     3
 1254.0     3
 2024.0     3
 315.0      3
 2434.0     3
 3573.0     3
 447.0      3
 1841.0     3
 2866.0     2
 3771.0     2
 12913.0    2
 487.0      2
 2869.0     2
 8845.0     2
 3142.0     2
 5704.0     2
 3201.0     2
 6951.0     2
 3687.0     2
 2657.0     2
 2969.0     2
           ..
 2058.0     1
 25090.0    1
 430.0      1
 1114.0     1
 482.0      1
 495.0      1
 780.0      1
 28881.0    1
 7058.0     1
 18548.0    1
 5542.0     1
 39298.0    1
 387.0      1
 131.0      1
 23098.0    1
 325.0      1
 2251.0     1
 466.0      1
 46755.0    1
 7051.0     1
 5295.0     1
 39996.0    1
 19922.0    1
 12040.0    1
 8788.0     1
 2870.0     1
 16393.0    1
 3599.0     1
 1671.0     1
 13684.0    1
Name: favorite_count, Length: 2135, dtype: int64

In [37]:
tweet_json_cleaned[tweet_json_cleaned['favorite_count'].isnull()]

,Unnamed: 0,Unnamed: 0.1,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,Unnamed: 0.1.1,favorite_count,retweet_count,stage,breed_predict
19,19,19,888202515573088257,NaN,NaN,2017-07-21 01:02:36,Twitter for iPhone,RT @dog_rates: This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX,887473957103951872,4196983835,2017-07-19 00:47:34,"https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1",13,10,Canela,None,None,None,None,https://pbs.twimg.com/media/DFDw2tyUQAAAFke.jpg,2.0,Pembroke,0.809197,True,Rhodesian_ridgeback,0.054950,True,beagle,0.038915,True,19,NaN,NaN,None,Pembroke
95,95,95,873697596434513921,NaN,NaN,2017-06-11 00:25:14,Twitter for iPhone,RT @dog_rates: This is Walter. He won't start hydrotherapy without his favorite floatie. 14/10 keep it pup Walter https://t.co/r28jFx9uyF,868880397819494400,4196983835,2017-05-28 17:23:24,"https://twitter.com/dog_rates/status/868880397819494401/photo/1,https://twitter.com/dog_rates/status/868880397819494401/photo/1",14,10,Walter,None,None,None,None,https://pbs.twimg.com/media/DA7iHL5U0AA1OQo.jpg,1.0,laptop,0.153718,False,French_bulldog,0.099984,True,printer,0.077130,False,95,NaN,NaN,None,French_bulldog
118,118,118,869988702071779329,NaN,NaN,2017-05-31 18:47:24,Twitter for iPhone,RT @dog_rates: We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10…,859196978902773760,4196983835,2017-05-02 00:04:57,https://twitter.com/dog_rates/status/859196978902773760/video/1,12,10,quite,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118,NaN,NaN,None,NaN
155,155,155,861769973181624320,NaN,NaN,2017-05-09 02:29:07,Twitter for iPhone,"RT @dog_rates: ""Good afternoon class today we're going to learn what makes a good boy so good"" 13/10 https://t.co/f1h2Fsalv9",806629075125202944,4196983835,2016-12-07 22:38:52,"https://twitter.com/dog_rates/status/806629075125202948/photo/1,https://twitter.com/dog_rates/status/806629075125202948/photo/1,https://twitter.com/dog_rates/status/806629075125202948/photo/1,https://twitter.com/dog_rates/status/806629075125202948/photo/1",13,10,None,None,None,None,None,https://pbs.twimg.com/media/CzG425nWgAAnP7P.jpg,2.0,Arabian_camel,0.366248,False,house_finch,0.209852,False,cocker_spaniel,0.046403,True,155,NaN,NaN,None,cocker_spaniel
260,260,260,842892208864923648,NaN,NaN,2017-03-18 00:15:37,Twitter for iPhone,RT @dog_rates: This is Stephan. He just wants to help. 13/10 such a good boy https://t.co/DkBYaCAg2d,807106840509214720,4196983835,2016-12-09 06:17:20,"https://twitter.com/dog_rates/status/807106840509214720/video/1,https://twitter.com/dog_rates/status/807106840509214720/video/1",13,10,Stephan,None,None,None,None,https://pbs.twimg.com/ext_tw_video_thumb/807106774843039744/pu/img/8XZg1xW35Xp2J6JW.jpg,1.0,Chihuahua,0.505370,True,Pomeranian,0.120358,True,toy_terrier,0.077008,True,260,NaN,NaN,None,Chihuahua
566,566,566,802247111496568832,NaN,NaN,2016-11-25 20:26:31,Twitter for iPhone,RT @dog_rates: Everybody drop what you're doing and look at this dog. 13/10 must be super h*ckin rare https://t.co/I1bJUzUEW5,779056095788752896,4196983835,2016-09-22 20:33:42,"https://twitter.com/dog_rates/status/779056095788752897/photo/1,https://twitter.com/dog_rates/status/779056095788752897/photo/1,https://twitter.com/dog_rates/status/779056095788752897/photo/1,https://twitter.com/dog_rates/status/779056095788752897/photo/1",13,10,None,None,None,None,None,https://pbs.twimg.com/media/Cs_DYr1XEAA54Pu.jp

Remake rating_numerator and rating_denominator.

First, change the type of rating_numerator and rating_denominator into float.

In [38]:
tweet_json_cleaned['rating_numerator']=tweet_json_cleaned['rating_numerator'].astype(float)
tweet_json_cleaned['rating_denominator']=tweet_json_cleaned['rating_denominator'].astype(float)

In [39]:

for index,row in tweet_json_cleaned.iterrows():
            rating=[]
            rating=re.findall(r'(\d+(\.\d+)?)\/(\d+)',row['text'],flags=0)
            if rating !=[]:
                if len(rating)>1 and rating[0][-1]!='10':
                          tweet_json_cleaned.set_value(index,'dog_count','multiple')
                else:
                          tweet_json_cleaned.set_value(index,'dog_count','single')
                        
                tweet_json_cleaned.set_value(index,'rating_numerator',rating[-1][0])
                tweet_json_cleaned.set_value(index,'rating_denominator',rating[-1][-1])
            else:
                tweet_json_cleaned.set_value(index,'rating_numerator',np.nan)
                tweet_json_cleaned.set_value(index,'rating_denominator',np.nan)

In [40]:
print(tweet_json_cleaned['rating_numerator'].value_counts(dropna=False))
print(tweet_json_cleaned['rating_denominator'].value_counts(dropna=False))

12.00      557
11.00      467
10.00      454
13.00      355
9.00       156
8.00       104
7.00        56
14.00       56
5.00        34
6.00        33
3.00        19
4.00        17
2.00        10
1.00         9
0.00         2
15.00        2
9.75         2
420.00       2
84.00        1
24.00        1
13.50        1
143.00       1
80.00        1
99.00        1
182.00       1
165.00       1
45.00        1
204.00       1
1776.00      1
666.00       1
11.27        1
121.00       1
11.26        1
88.00        1
144.00       1
9.50         1
20.00        1
44.00        1
60.00        1
Name: rating_numerator, dtype: int64
10.0     2340
50.0        2
80.0        2
150.0       1
110.0       1
90.0        1
130.0       1
70.0        1
170.0       1
120.0       1
16.0        1
20.0        1
40.0        1
7.0         1
15.0        1
Name: rating_denominator, dtype: int64


Remake dog names

In [41]:
for index,row in tweet_json_cleaned.iterrows():
            text=row['text']
            
            if re.match(r'.*This is [A-Z][A-Za-zñáéíóúü/-]+([.,!?\s]|( (and|(&amp;)) [A-Z][A-Za-zñáéíóúü/-]+))',text):
                
                name=re.findall(r'.*This is ([A-Z][A-Za-zñáéíóúü/-]+)[.,!?\s]',text)
                name2=re.findall(r'.* (and|(&amp;)) ([A-Z][A-Za-zñáéíóúü/-]+)',text)
                tweet_json_cleaned.set_value(index,'new_name',name[0])
            
                if name2 !=[]:
                    print(name2)
                    tweet_json_cleaned.set_value(index,'dog_count','multiple')

            elif re.match(r'.*Meet [A-Z][A-Za-zñáéíóúü/-]+([.,!?\s]|( (and|(&amp;)) [A-Z][A-Za-zñáéíóúü/-]+))',text):
                
                name=re.findall(r'.*Meet ([A-Z][A-Za-zñáéíóúü/-]+)[.,!?\s]',text)  
                name2=re.findall(r'.* (and|(&amp;)) ([A-Z][A-Za-zñáéíóúü/-]+)',text)
                tweet_json_cleaned.set_value(index,'new_name',name[0])
                
                if name2 !=[]:
                    print(name2)
                    tweet_json_cleaned.set_value(index,'dog_count','multiple')
               
            elif re.match(r'.*Say hello to [A-Z][A-Za-zñáéíóúü/-]+([.,!?]|( (and|(&amp;)) [A-Z][A-Za-zñáéíóúü/-]+))',text):
                
                name=re.findall(r'.*Say hello to ([A-Z][A-Za-zñáéíóúü/-]+)[.,!?\s]',text)
                name2=re.findall(r'.* (and|(&amp;)) ([A-Z][A-Za-zñáéíóúü/-]+)',text)
                tweet_json_cleaned.set_value(index,'new_name',name[0])
                if name2 !=[]:
                    print(name2)
                    tweet_json_cleaned.set_value(index,'dog_count','multiple')
                
            elif re.match(r'.*named [A-Z][A-Za-zñáéíóúü/-]+([.,!?\s]|( (and|(&amp;)) [A-Z][A-Za-zñáéíóúü/-]+))',text):
                
                name=re.findall(r'.*named ([A-Z][A-Za-zñáéíóúü/-]+)[.,!?\s]',text)
                name2=re.findall(r'.* (and|(&amp;)) ([A-Z][A-Za-zñáéíóúü/-]+)',text)
    
                tweet_json_cleaned.set_value(index,'new_name',name[0])
                if name2 !=[]:
                    print(name2)
                    tweet_json_cleaned.set_value(index,'dog_count','multiple')
                    
            elif re.match(r'.*name is [A-Z][A-Za-zñáéíóúü/-]+([.,!?\s]|( (and|(&amp;)) [A-Z][A-Za-zñáéíóúü/-]+))',text):
                
                name=re.findall(r'.*name is ([A-Z][A-Za-zñáéíóúü/-]+)[.,!?\s]',text)
                name2=re.findall(r'.* (and|(&amp;)) ([A-Z][A-Za-zñáéíóúü/-]+)',text)
                
                tweet_json_cleaned.set_value(index,'new_name',name[0])
                if name2 !=[]:
                    print(name2)
                    tweet_json_cleaned.set_value(index,'dog_count','multiple')    
            else:
                tweet_json_cleaned.set_value(index,'new_name',None)

                


[('and', '', 'Hank')]
[('and', '', 'Oliver')]
[('and', '', 'Morple')]
[('and', '', 'Daisy')]
[('and', '', 'Gunner')]
[('and', '', 'Gunner')]
[('&amp;', '&amp;', 'Harold')]
[('and', '', 'Jersey')]
[('and', '', 'Elsa')]
[('&amp;', '&amp;', 'Patti')]
[('&amp;', '&amp;', 'Wilbur')]
[('and', '', 'Harley')]
[('and', '', 'Gary')]
[('and', '', 'Murphy')]
[('and', '', 'Ty')]
[('and', '', 'Charlie')]
[('and', '', 'Ty')]
[('&amp;', '&amp;', 'Lila')]
[('&amp;', '&amp;', 'Honey')]
[('&amp;', '&amp;', 'Jil')]
[('and', '', 'Seaweed')]
[('and', '', 'Thumpelina')]
[('and', '', 'Jupiter')]
[('and', '', 'Millie')]
[('&amp;', '&amp;', 'Murphy')]
[('and', '', 'Sophie')]
[('and', '', 'Flurp')]
[('and', '', 'Clarkus')]
[('and', '', 'Roxy')]
[('and', '', 'Ruby')]
[('and', '', 'Amos')]
[('&amp;', '&amp;', 'Kiwi')]
[('&amp;', '&amp;', 'Wilbur')]
[('&amp;', '&amp;', 'Ruffalo')]
[('and', '', 'Plip')]
[('and', '', 'Taco')]
[('and', '', 'Comcast')]
[('&amp;', '&amp;', 'Charlie')]
[('and', '', 'Izzy')]
[('&amp;', '&

In [42]:
tweet_json_cleaned[tweet_json_cleaned['new_name'].isnull()].to_csv('new_name.csv')

In [43]:
tweet_json_cleaned['new_name'].value_counts(dropna=False)

NaN           832
Charlie        12
Cooper         11
Oliver         11
Lucy           11
Lola           10
Penny          10
Tucker         10
Bo              9
Winston         9
Sadie           8
Bailey          7
Toby            7
Buddy           7
Daisy           7
Oscar           6
Leo             6
Milo            6
Rusty           6
Bella           6
Koda            6
Stanley         6
Jax             6
Scout           6
Finn            5
Sammy           5
Oakley          5
Alfie           5
Bentley         5
Sunny           5
             ... 
Maks            1
Bradlay         1
Erik            1
Brandonald      1
Crumpet         1
Eazy-E          1
Mutt            1
Alfonso         1
Thor            1
Grady           1
Adele           1
Harrison        1
Grizz           1
Tino            1
Enchilada       1
Glacier         1
Claude          1
Theo            1
Dale            1
Grizzwald       1
Kloey           1
Arlen           1
Meera           1
Lilah           1
Cora      

Remove tweets without a jpg_url.

Remove record with multiple dog_count because the model only considers and predicts one dog.

Remove column "unnamed: 0" "name" and "dog_count"

In [44]:
tweet_json_cleaned=tweet_json_cleaned[tweet_json_cleaned['jpg_url'].notnull()]
print(tweet_json_cleaned.shape)
tweet_json_cleaned=tweet_json_cleaned[tweet_json_cleaned['dog_count']=='single']
print(tweet_json_cleaned.shape)
del tweet_json_cleaned['name']
del tweet_json_cleaned['dog_count']
del tweet_json_cleaned['Unnamed: 0.1']
del tweet_json_cleaned['doggo']
del tweet_json_cleaned['floofer']
del tweet_json_cleaned['puppo']
del tweet_json_cleaned['pupper']
del tweet_json_cleaned['Unnamed: 0.1.1']
del tweet_json_cleaned['Unnamed: 0']

(2075, 37)
(2022, 37)


In [45]:
tweet_json_cleaned.shape

(2022, 29)

In [46]:
tweet_json_cleaned.sample(3)

,Unnamed: 0,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,favorite_count,retweet_count,stage,breed_predict,new_name
1966,1966,673343217010679808,NaN,NaN,2015-12-06 03:28:27,Twitter for iPhone,IT'S SO SMALL ERMERGERF 11/10 https://t.co/dNUbKOSiWW,NaN,NaN,NaT,https://twitter.com/dog_rates/status/673343217010679808/photo/1,11.0,10.0,https://pbs.twimg.com/media/CVgyFSyU4AA9p1e.jpg,1.0,Chihuahua,0.541408,True,Italian_greyhound,0.156891,True,miniature_pinscher,0.069556,True,1025.0,288.0,None,Chihuahua,None
1573,1573,687494652870668288,NaN,NaN,2016-01-14 04:41:12,Twitter for iPhone,This is Marq. He stole this car. 7/10 wtf Marq? https://t.co/MHScqo5l8c,NaN,NaN,NaT,https://twitter.com/dog_rates/status/687494652870668288/photo/1,7.0,10.0,https://pbs.twimg.com/media/CYp4vFrVAAEs9AX.jpg,1.0,Rottweiler,0.391471,True,miniature_pinscher,0.273595,True,Tibetan_mastiff,0.041692,True,2087.0,650.0,None,Rottweiler,Marq
1336,1336,705223444686888960,NaN,NaN,2016-03-03 02:49:06,Twitter for iPhone,This is Bode. He's a heavy sleeper. 9/10 https://t.co/YMkxhGWUqv,NaN,NaN,NaT,https://twitter.com/dog_rates/status/705223444686888960/photo/1,9.0,10.0,https://pbs.twimg.com/media/Ccl0-HVVAAAf8aK.jpg,1.0,Egyptian_cat,0.090508,False,Chesapeake_Bay_retriever,0.077373,True,Mexican_hairless,0.049472,True,2796.0,892.0,None,Chesapeake_Bay_retriever,Bode


In [47]:
tweet_json_cleaned.to_csv('twitter_archive_master.csv')